Import the MicroGP package (first install it: ``!pip install microgp``)


In [1]:
import sys
import microgp as ugp
ugp.logging.DefaultLogger.setLevel(level=ugp.logging.INFO)

M:\LuBa\Documenti\GitHub\microgp4\src\microgp\__init__.py:51: UserWarning: Paranoia checks are active: performances can be significantly impaired (consider using '-O')
  warnings.warn(WARN_DBG, UserWarning)
11:45:08 INFO Initialized random generator


### Creation of the individuals structure
Define parameters

In [2]:
reg_alternatives = ['%eax', '%ebx', '%ecx', '%edx']
reg_param = ugp.make_parameter(ugp.parameter.Categorical, alternatives=reg_alternatives)
instr_alternatives = ['add', 'sub', 'and', 'or', 'xor', 'cmp']
instr_param = ugp.make_parameter(ugp.parameter.Categorical, alternatives=instr_alternatives)
shift_alternatives = ['shr', 'shl']
shift_param = ugp.make_parameter(ugp.parameter.Categorical, alternatives=shift_alternatives)
jmp_alternatives = ['ja', 'jz', 'jnz', 'je', 'jne', 'jc', 'jnc', 'jo', 'jno', 'jmp']
jmp_instructions = ugp.make_parameter(ugp.parameter.Categorical, alternatives=jmp_alternatives)
integer = ugp.make_parameter(ugp.parameter.Integer, min=-32768, max=32767)
int8 = ugp.make_parameter(ugp.parameter.Integer, min=0, max=256)
jmp_target = ugp.make_parameter(ugp.parameter.LocalReference,
                                allow_self=False,
                                allow_forward=True,
                                allow_backward=False,
                                frames_up=0)

Define the macros

In [3]:
jmp1 = ugp.Macro("    {jmp_instr} {jmp_ref}", {'jmp_instr': jmp_instructions, 'jmp_ref': jmp_target})
instr_op_macro = ugp.Macro("    {instr} {regS}, {regD}",{'instr': instr_param, 'regS': reg_param, 'regD': reg_param})
shift_op_macro = ugp.Macro("    {shift} ${int8}, {regD}", {'shift': shift_param, 'int8': int8, 'regD': reg_param})
prologue_macro = ugp.Macro('    .file   "solution.c"\n' +
                           '    .text\n' +
                           '    .globl  _darwin\n' +
                           '    .def    _darwin;        .scl    2;      .type   32;     .endef\n' +
                           '_darwin:\n' +
                           'LFB17:\n' +
                           '    .cfi_startproc\n' +
                           '    pushl   %ebp\n' +
                           '    .cfi_def_cfa_offset 8\n' +
                           '    .cfi_offset 5, -8\n' +
                           '    movl    %esp, %ebp\n' +
                           '    .cfi_def_cfa_register 5\n')
init_macro = ugp.Macro("    movl	${int_a}, %eax\n" +
                       "    movl	${int_b}, %ebx\n" +
                       "    movl	${int_c}, %ecx\n" +
                       "    movl	${int_d}, %edx\n",
                       {'int_a': integer, 'int_b': integer, 'int_c': integer, 'int_d': integer})
epilogue_macro = ugp.Macro(
    '    movl	%eax, -4(%ebp)\n' +
    '    movl	-4(%ebp), %eax\n' +
    '    leave\n' +
    '    .cfi_restore 5\n' +
    '    .cfi_def_cfa 4, 4\n' +
    '    ret\n' +
    '    .cfi_endproc\n' +
    'LFE17:\n' +
    '   .ident  "GCC: (MinGW.org GCC-8.2.0-5) 8.2.0"\n')

Define section

In [4]:
sec1 = ugp.make_section({jmp1, instr_op_macro, shift_op_macro}, size=(1, 50))

Create an instruction library

In [5]:
library = ugp.Constraints(file_name="solution{id}.s")

Define the sections in the library

In [6]:
library['main'] = [prologue_macro, init_macro, sec1, epilogue_macro]

### Define the evaluator script and the fitness type

In [7]:
if sys.platform != "win32":
    exit(-1)
else:
    script = "eval.bat"
library.evaluator = ugp.fitness.make_evaluator(evaluator=script, fitness_type=ugp.fitness.Lexicographic)

Define and set a property. It checks whether the section 'sec1' has or not the same number of 'shr' and 'shl'

In [8]:
def shift_count(individual, frame, **kk):
    from microgp.individual import get_nodes_in_frame
    shl_count = 0
    shr_count = 0
    nodes = get_nodes_in_frame(individual, frame)
    for node in nodes:
        parameters = individual.nodes[node]['parameters']
        if 'shift' in parameters.keys():
            if parameters['shift'].value == 'shr':
                shr_count += 1
            elif parameters['shift'].value == 'shl':
                shl_count += 1
    return {'shl_count': shl_count, 'shr_count': shr_count}

sec1.properties.add_base_builder(shift_count)
sec1.properties.add_checker(lambda shl_count, shr_count, **v: shl_count == shr_count)

### Create a list of operators with their aritiy

In [9]:
operators = ugp.Operators()

Add initialization operators

In [10]:
operators += ugp.GenOperator(ugp.create_random_individual, 0)

Add mutation operators

In [11]:
operators += ugp.GenOperator(ugp.hierarchical_mutation, 1)
operators += ugp.GenOperator(ugp.flat_mutation, 1)
operators += ugp.GenOperator(ugp.add_node_mutation, 1)
operators += ugp.GenOperator(ugp.remove_node_mutation, 1)

Add crossover operators

In [12]:
operators += ugp.GenOperator(ugp.macro_pool_one_cut_point_crossover, 2)
operators += ugp.GenOperator(ugp.macro_pool_uniform_crossover, 2)

Create the object that will manage the evolution

In [13]:
mu = 10
nu = 20
sigma = 0.7
lambda_ = 7
max_age = 10

darwin = ugp.Darwin(
    constraints=library,
    operators=operators,
    mu=mu,
    nu=nu,
    lambda_=lambda_,
    sigma=sigma,
    max_age=max_age,
)

Evolve (create random individuals and use selected genetic operators to evolve them)

In [14]:
darwin.evolve()

M:\LuBa\Documenti\GitHub\microgp4\src\microgp\parameter\helpers.py:64: RuntimeWarning: invalid value encountered in true_divide
  probs = probs / sum(probs)


Print best individuals, and their graph

In [15]:
ugp.print_individual(darwin.archive.individuals,
                     msg="These are the best ever individuals:",
                     plot=True, score=True)

11:45:18 These are the best ever individuals:
11:45:18     .file   "solution.c"
11:45:18     .text
11:45:18     .globl  _darwin
11:45:18     .def    _darwin;        .scl    2;      .type   32;     .endef
11:45:18 _darwin:
11:45:18 LFB17:
11:45:18     .cfi_startproc
11:45:18     pushl   %ebp
11:45:18     .cfi_def_cfa_offset 8
11:45:18     .cfi_offset 5, -8
11:45:18     movl    %esp, %ebp
11:45:18     .cfi_def_cfa_register 5
11:45:18 
11:45:18     movl	$-12578, %eax
11:45:18     movl	$31294, %ebx
11:45:18     movl	$-3567, %ecx
11:45:18     movl	$24530, %edx
11:45:18 
11:45:18     jno n4
11:45:18 n4:
11:45:18     movl	%eax, -4(%ebp)
11:45:18     movl	-4(%ebp), %eax
11:45:18     leave
11:45:18     .cfi_restore 5
11:45:18     .cfi_def_cfa 4, 4
11:45:18     ret
11:45:18     .cfi_endproc
11:45:18 LFE17:
11:45:18    .ident  "GCC: (MinGW.org GCC-8.2.0-5) 8.2.0"
M:\LuBa\Documenti\GitHub\microgp4\src\microgp\individual_operators.py:66: RuntimeWarning: Can't plot individual without ``matplotlib''
